# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('data.sqlite')
cur = con.cursor()

In [3]:
pd.read_sql('''
SELECT *
FROM employees
LIMIT 5
''', con)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
cur.execute("""
SELECT city, employees.firstName, employees.lastName
FROM employees
JOIN offices
    ON employees.officeCode = offices.officeCode
WHERE offices.city = 'Boston'
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,city,firstName,lastName
0,Boston,Julie,Firrelli
1,Boston,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [5]:
# Your code here

cur.execute("""
SELECT offices.officeCode, offices.city, COUNT(employees.lastName) AS 'number of employees'
FROM employees
JOIN offices
    ON employees.officeCode = offices.officeCode
GROUP BY offices.officeCode
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,officeCode,city,number of employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2


In [47]:
cur.execute("""
SELECT count(status), customers.city
FROM orders
JOIN customers
    USING(customerNumber)
WHERE orders.status = "On Hold"
GROUP BY customers.city
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,count(status),city
0,1,Boston
1,1,Luleå
2,1,Newark
3,1,San Jose


In [7]:
cur.execute("""
SELECT count(status)
FROM orders
WHERE status = "Shipped"
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,count(status)
0,303


## Write 3 questions of your own and answer them

In [29]:
cur.execute("""
select status, count('Shipped')
from orders
group by status
""").fetchall()

[('Cancelled', 6),
 ('Disputed', 3),
 ('In Process', 6),
 ('On Hold', 4),
 ('Resolved', 4),
 ('Shipped', 303)]

In [8]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""
# Example question answer:
q = """
SELECT
    o.officeCode,
    o.city,
    COUNT(c.customerNumber) AS n_customers
FROM offices AS o
JOIN employees AS e
    USING(officeCode)
JOIN customers AS c
    ON e.employeeNumber = c.salesRepEmployeeNumber
GROUP BY officeCode
;
"""
pd.read_sql(q, con)


,officeCode,city,n_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [45]:
"""
Question 1
How many orders shipped per office?
"""

# Your code here
cur.execute("""
SELECT offices.city, Count(status)
FROM offices 
JOIN orders
WHERE orders.status = 'Shipped'
GROUP BY offices.officeCode

""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,city,Count(status)
0,San Francisco,303
1,Boston,303
2,NYC,303
3,Paris,303
4,Tokyo,303
5,Sydney,303
6,London,303
7,Boston,303


In [55]:
"""
How many orders in a office has order status In Process?
"""

# Your code here
cur.execute("""
SELECT count(status), customers.city
FROM orders
JOIN customers
    USING(customerNumber)
WHERE orders.status = "Shipped"
GROUP BY customers.city
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df


,count(status),city
0,3,Allentown
1,2,Auckland
2,8,Auckland
3,3,Barcelona
4,3,Bergamo
...,...,...
73,2,Vancouver
74,2,Versailles
75,3,Wellington
76,2,White Plains


In [54]:
"""
Which customers have orders on hold?
"""

# Your code here

cur.execute("""
SELECT count(status), customers.city, customers.contactLastName
FROM orders
JOIN customers
    USING(customerNumber)
WHERE orders.status = "On Hold"
GROUP BY customers.city
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,count(status),city,contactLastName
0,1,Boston,Yoshido
1,1,Luleå,Berglund
2,1,Newark,Brown
3,1,San Jose,Frick


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [62]:
# Your code here
cur.execute("""
SELECT employees.lastName, employees.firstName, products.productName
FROM employees
JOIN customers
    ON customers.salesRepEmployeeNumber = employees.employeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
JOIN products
    USING(productCode)
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,lastName,firstName,productName
0,Jennings,Leslie,1958 Setra Bus
1,Jennings,Leslie,1940 Ford Pickup Truck
2,Jennings,Leslie,1939 Cadillac Limousine
3,Jennings,Leslie,1996 Peterbilt 379 Stake Bed with Outrigger
4,Jennings,Leslie,1968 Ford Mustang
...,...,...,...
2991,Gerard,Martin,1954 Greyhound Scenicruiser
2992,Gerard,Martin,1950's Chicago Surface Lines Streetcar
2993,Gerard,Martin,Diamond T620 Semi-Skirted Tanker
2994,Gerard,Martin,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [65]:
# Your code here
cur.execute("""
SELECT firstName, lastName, SUM(quantityOrdered) 
FROM employees
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY lastName
ORDER BY lastName
""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df


,firstName,lastName,SUM(quantityOrdered)
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [66]:
# Your code here
cur.execute("""
SELECT firstName, lastName, COUNT(productCode) AS Unique_Products_Sold
FROM employees
JOIN customers
    ON employees.employeeNumber = customers.salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY lastName
HAVING Unique_Products_Sold >200
ORDER BY lastName

""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,firstName,lastName,Unique_Products_Sold
0,Larry,Bott,236
1,Pamela,Castillo,272
2,Gerard,Hernandez,396
3,Leslie,Jennings,331
4,Barry,Jones,220
5,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!